# First steps for Google Places API

**To Do**

- seit wann Gemini Modell

Meine Aufgaben
- Datensatz vorbereiten: Tübingen
- zwei Datensätze: Name und Place_id + alle weiteren infos


## Preparations

In [ ]:
#!pip install -r requirements.txt

In [1]:
import requests
from urllib.parse import urlencode
import googlemaps
import os
from dotenv import load_dotenv
import time
import pandas as pd

In [2]:
# import the api_key from the api_key.py file
from api_key import api_key

## 1. nearbysearch: only location

In [5]:
def get_place_info(address):
# Base URL
  base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
# Parameters in a dictionary
  params = {
   "input": address,
   "inputtype": "textquery",
   "fields": "name,geometry/location,place_id",
   "key": api_key,
  }
# Send request and capture response
  response = requests.get(base_url, params=params)
# Check if the request was successful
  if response.status_code == 200:
    return response.json()
  else:
    return None

In [6]:
info = get_place_info("tübingen")	
print(info)

longitude =  info["candidates"][0].get("geometry", {}).get("location", {}).get("lng")
latitude = info["candidates"][0].get("geometry", {}).get("location", {}).get("lat")

{'candidates': [{'geometry': {'location': {'lat': 48.5216364, 'lng': 9.0576448}}, 'name': 'Tübingen', 'place_id': 'ChIJgdDN7dT6mUcRjacz_s6uCKw'}], 'status': 'OK'}


In [61]:
# all fields that are possible and interesting for us
field_nearby_places = [
    # Fields that trigger the Nearby Search (Basic) SKU
    "places.accessibilityOptions", "places.addressComponents", "places.adrFormatAddress", 
    "places.attributions", "places.businessStatus", "places.containingPlaces", 
    "places.displayName", "places.formattedAddress", "places.googleMapsLinks", 
    "places.id", "places.location", "places.name", "places.plusCode", 
    "places.primaryType", "places.primaryTypeDisplayName", "places.pureServiceAreaBusiness", 
    "places.shortFormattedAddress", "places.subDestinations", "places.types", 
    "places.utcOffsetMinutes", "places.viewport",

    # Fields that trigger the Nearby Search (Advanced) SKU
    "places.currentOpeningHours", "places.currentSecondaryOpeningHours", 
    "places.nationalPhoneNumber", "places.priceLevel", 
    "places.priceRange", "places.rating", "places.regularOpeningHours", 
    "places.regularSecondaryOpeningHours", "places.userRatingCount", "places.websiteUri",

    # Fields that trigger the Nearby Search (Preferred) SKU
    "places.allowsDogs", "places.curbsidePickup", "places.delivery", "places.dineIn", 
    "places.editorialSummary", "places.evChargeOptions", "places.fuelOptions", 
    "places.goodForChildren", "places.goodForGroups", "places.goodForWatchingSports", 
    "places.liveMusic", "places.menuForChildren", "places.parkingOptions", 
    "places.paymentOptions", "places.outdoorSeating", "places.reservable", "places.restroom", 
    "places.reviews", "places.servesBeer", 
    "places.servesBreakfast", "places.servesBrunch", "places.servesCocktails", 
    "places.servesCoffee", "places.servesDessert", "places.servesDinner", 
    "places.servesLunch", "places.servesVegetarianFood", "places.servesWine", 
    "places.takeout"
]


def get_nearby_places(latitude, longitude, radius=500, max_results=10):
    """
    Calls the Google Places API to get nearby places within a specified radius.    
    
    Args:
        latitude (float): Latitude of the center of the search area.
        longitude (float): Longitude of the center of the search area.
        radius (float): Search radius in meters (default 500).
        max_results (int): Maximum number of results to return (default is 10, range 1 - 20 possible).
    
    Returns:
        dict: A dictionary containing the search results.
    """

    # Base URL
    base_url = "https://places.googleapis.com/v1/places:searchNearby"
    # JSON request body
    body = {"includedTypes": ["restaurant"],  # Restrict to restaurants only
            "maxResultCount": max_results,  # Max number of results to return
            "locationRestriction": {
                "circle": {
                    "center": {"latitude": latitude,
                               "longitude": longitude},
                    "radius": radius  # Radius for the search
                }
            },
            "languageCode": "en",  # Language code for the results
        }
    # Headers
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "places.name,places.id,places.displayName" 
        #"X-Goog-FieldMask": ",".join(field_nearby_places) 
    }
    # Send the POST request to the API
    response = requests.post(base_url, headers=headers, json=body)
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, "Message:", response.text)

In [62]:
result = get_nearby_places(latitude, longitude, max_results=15, radius=100)

## 2. textsearch: keywords + opt. location
https://developers.google.com/maps/documentation/places/web-service/text-search#text-search-requests

filtering options:
- includePureServiceAreaBusinesses: filter for physical business location true or false
- locationRestriction or locationBias --> if omited API uses IP biasing by default; but locationBias parameter can be overridden if the textQuery 
    - rectangular Viewport or as a circle
- minRating
- openNow
- priceLevels
- rankPreference
- regionCode



In [5]:
# all fields that are possible and interesting for us
field_textsearch = [
    # Fields that trigger the Text Search (ID Only) SKU
    "places.attributions", "places.id", "places.name", "nextPageToken",

    # Fields that trigger the Text Search (Basic) SKU
    "places.accessibilityOptions", "places.addressComponents", "places.adrFormatAddress", 
    "places.businessStatus", "places.containingPlaces", "places.displayName", 
    "places.formattedAddress", "places.googleMapsLinks", 
    "places.location", 
    "places.plusCode", "places.primaryType", "places.primaryTypeDisplayName", 
    "places.pureServiceAreaBusiness", "places.shortFormattedAddress", "places.subDestinations", 
    "places.types", "places.utcOffsetMinutes", "places.viewport",

    # Fields that trigger the Text Search (Advanced) SKU
    "places.currentOpeningHours", "places.currentSecondaryOpeningHours", 
    "places.internationalPhoneNumber", "places.nationalPhoneNumber", "places.priceLevel", 
    "places.priceRange", "places.rating", "places.regularOpeningHours", 
    "places.regularSecondaryOpeningHours", "places.userRatingCount", "places.websiteUri",

    # Fields that trigger the Text Search (Preferred) SKU
    "places.allowsDogs", "places.curbsidePickup", "places.delivery", "places.dineIn", 
    "places.editorialSummary", "places.evChargeOptions", "places.fuelOptions", 
    "places.goodForChildren", "places.goodForGroups", "places.goodForWatchingSports", 
    "places.liveMusic", "places.menuForChildren", "places.parkingOptions", 
    "places.paymentOptions", "places.outdoorSeating", "places.reservable", "places.restroom", 
    "places.reviews", "places.routingSummaries", "places.servesBeer", 
    "places.servesBreakfast", "places.servesBrunch", "places.servesCocktails", 
    "places.servesCoffee", "places.servesDessert", "places.servesDinner", 
    "places.servesLunch", "places.servesVegetarianFood", "places.servesWine", 
    "places.takeout"
]

In [6]:
def get_places_textsearch(query, radius=None, max_results=10, page_token=None):
    """
    Calls the Google Places Text Search API to get places matching a query.
    
    Args:
        query (str): The search query string, e.g., "restaurants in New York".
        latitude (float): Latitude of the center of the search area (optional).
        longitude (float): Longitude of the center of the search area (optional).
        radius (float): Search radius in meters (optional, default 500).
        max_results (int): Maximum number of results to return per page (default is 10, range 1-20).

    Returns:
        dict: A dictionary containing the search results.
    """

    # Base URL
    base_url = "https://places.googleapis.com/v1/places:searchText"
    # JSON request body
    body = {
        "includedType": "restaurant",  # Restrict to restaurants only
        "strictTypeFiltering": True,  # Only return results of the specified type
        "textQuery": query,
        "pageSize": max_results,  # Specify max results per page
        "pageToken": page_token,  # Token for the next page of results
        "languageCode": "en",  # Language code for the results
    }
    # Headers
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "places.name,places.id,places.displayName,nextPageToken" 
        #"X-Goog-FieldMask": ",".join(field_textsearch) 
    }
    # Send the POST request to the API
    response = requests.post(base_url, headers=headers, json=body)
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, "Message:", response.text)

In [7]:
res = get_places_textsearch("spicy vegetarian food in Nürnberg", max_results=5)
res

{'places': [{'name': 'places/ChIJQ2lIViZXn0cRi1v-yFyM0eg',
   'id': 'ChIJQ2lIViZXn0cRi1v-yFyM0eg',
   'displayName': {'text': 'My Hao Vietnamese Vegan Cuisine',
    'languageCode': 'en'}},
  {'name': 'places/ChIJuxfj8_RZn0cRYcFsf_c2IV8',
   'id': 'ChIJuxfj8_RZn0cRYcFsf_c2IV8',
   'displayName': {'text': 'Chay Vegan Kitchen - Nürnberg',
    'languageCode': 'de'}},
  {'name': 'places/ChIJzbN2OrtXn0cRoT4rD0rxuPs',
   'id': 'ChIJzbN2OrtXn0cRoT4rD0rxuPs',
   'displayName': {'text': 'Pure Food - The Vegan Café & Bar',
    'languageCode': 'de'}},
  {'name': 'places/ChIJ2bj6IKRXn0cRi1Ie02tjBX4',
   'id': 'ChIJ2bj6IKRXn0cRi1Ie02tjBX4',
   'displayName': {'text': 'Cat Temple Nurnberg', 'languageCode': 'en'}},
  {'name': 'places/ChIJ1yvx9YJXn0cRYbkaRqjL9j4',
   'id': 'ChIJ1yvx9YJXn0cRYbkaRqjL9j4',
   'displayName': {'text': 'Kaschemme Nürnberg', 'languageCode': 'de'}}],
 'nextPageToken': 'AdDdOWr_I78Vi8kt1sK00MaHN9iBBwqtHqECEtWHXsd2t2TWQCc6wALxJcIoNneRppcdHtFTeSxR6LMXw4eJ9KxWY1Dg1KV2TEOjITBTlIVwg

In [8]:
token = res['nextPageToken']
res2 = get_places_textsearch("spicy vegetarian food in Nürnberg", max_results=5, page_token=token)
res2

{'places': [{'name': 'places/ChIJN16uonpXn0cRKGkSGHhyGlA',
   'id': 'ChIJN16uonpXn0cRKGkSGHhyGlA',
   'displayName': {'text': 'Bodhi | Nürnberg', 'languageCode': 'de'}},
  {'name': 'places/ChIJZb_-SAdXn0cReEQyrvGmqtg',
   'id': 'ChIJZb_-SAdXn0cReEQyrvGmqtg',
   'displayName': {'text': 'Kaspar Schmauser Nürnberg', 'languageCode': 'de'}},
  {'name': 'places/ChIJ-96-t1VWn0cR0yW73XwWxyY',
   'id': 'ChIJ-96-t1VWn0cR0yW73XwWxyY',
   'displayName': {'text': 'Veganel', 'languageCode': 'en'}},
  {'name': 'places/ChIJaRG4P6RXn0cRnWR40M2y-fg',
   'id': 'ChIJaRG4P6RXn0cRnWR40M2y-fg',
   'displayName': {'text': 'Zeit & Raum - vegetarisch & vegan',
    'languageCode': 'de'}},
  {'name': 'places/ChIJGSyLd9BXn0cRggtocihIwdk',
   'id': 'ChIJGSyLd9BXn0cRggtocihIwdk',
   'displayName': {'text': 'Shiva Restaurant Nürnberg',
    'languageCode': 'en'}}],
 'nextPageToken': 'AdDdOWoMt5fT925kUnFUQ2GTH3HzhZIUIRPiLYWdRmcKQdt4K5Up4Vuk5X_F25vUN99B9yrng3VdhIIkfSyYBnKt_r1HPRc_m2_vk91vpz6DCoxzAIl7nTgTJ2eJVtb5D2fUarQg

## 3. place details
https://developers.google.com/maps/documentation/places/web-service/place-details 


for later:  Generate Google Maps link
f"https://www.google.com/maps/place/?q=place_id:{place_id}"
& under 'googleMapsLinks'\'reviewsUri'

In [31]:
field_place_details = ["id", "displayName", "name", "addressComponents", "adrFormatAddress", "formattedAddress","location", 
                    "types", "accessibilityOptions", "businessStatus", "containingPlaces", "displayName", "googleMapsLinks", 
                    "primaryType", "primaryTypeDisplayName", "pureServiceAreaBusiness", "subDestinations", "currentOpeningHours", 
                    "nationalPhoneNumber", "priceLevel", "priceRange", "rating", "regularOpeningHours", "regularSecondaryOpeningHours", "userRatingCount", 
                    "websiteUri",
                    "allowsDogs", "curbsidePickup", "delivery", "dineIn", "editorialSummary", "evChargeOptions", "fuelOptions", 
                    "goodForChildren", "goodForGroups", "goodForWatchingSports", "liveMusic", "menuForChildren", "parkingOptions", 
                    "paymentOptions", "outdoorSeating", "reservable", "restroom", "reviews", "servesBeer", "servesBreakfast", 
                    "servesBrunch", "servesCocktails", "servesCoffee", "servesDessert", "servesDinner", "servesLunch", 
                    "servesVegetarianFood", "servesWine", "takeout"]

def get_place_details(place_id):
    """
    Calls the Google Places API to get details about a place.
    
    Args:
        place_id (str): A text query for the place.
    
    Returns:
        dict: The JSON response from the API.
    """

    # Base URL
    base_url = f"https://places.googleapis.com/v1/places/{place_id}"
    # Query parameters
    params = {
        "languageCode": "en"  # This is a query parameter, not a header
    }
    # Headers
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": ",".join(field_place_details) 
    }
    # Send request and capture response
    response = requests.get(base_url, headers=headers, params=params)
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, "Message:", response.text)

In [35]:
details = get_place_details("ChIJGTtrv9P6mUcRtLRexsK4n1E")
details

{'name': 'places/ChIJGTtrv9P6mUcRtLRexsK4n1E',
 'id': 'ChIJGTtrv9P6mUcRtLRexsK4n1E',
 'types': ['italian_restaurant',
  'restaurant',
  'food',
  'point_of_interest',
  'establishment'],
 'nationalPhoneNumber': '07071 25157',
 'formattedAddress': 'Bursagasse, 72070 Tübingen, Germany',
 'addressComponents': [{'longText': 'Bursagasse',
   'shortText': 'Bursagasse',
   'types': ['route'],
   'languageCode': 'de'},
  {'longText': 'Tübingen',
   'shortText': 'Tübingen',
   'types': ['locality', 'political'],
   'languageCode': 'de'},
  {'longText': 'Tübingen',
   'shortText': 'Tübingen',
   'types': ['administrative_area_level_3', 'political'],
   'languageCode': 'de'},
  {'longText': 'Tübingen',
   'shortText': 'TÜ',
   'types': ['administrative_area_level_2', 'political'],
   'languageCode': 'de'},
  {'longText': 'Baden-Württemberg',
   'shortText': 'BW',
   'types': ['administrative_area_level_1', 'political'],
   'languageCode': 'de'},
  {'longText': 'Germany',
   'shortText': 'DE',
   

# load dataset for all restaurants in Tübingen

In [8]:
# functon to divide the area into grid cells
def divide_area(low_lat, low_long, high_lat, high_long, step_size):
    """
    Divides a rectangular area into a grid of smaller rectangles.
    
    Args:
        low_lat (float): The latitude of the lower-left corner.
        low_long (float): The longitude of the lower-left corner.
        high_lat (float): The latitude of the upper-right corner.
        high_long (float): The longitude of the upper-right corner.
        step_size (float): The size of the grid cells in degrees.
    
    Returns:
        list: A list of tuples containing the coordinates of the grid cells.
    """
    # Calculate the number of steps in latitude and longitude
    steps_lat = int((high_lat - low_lat) / step_size)
    steps_long = int((high_long - low_long) / step_size)
    # Create a list of tuples with the coordinates of the grid cells
    grid = []
    for i in range(steps_lat):
        for j in range(steps_long):
            # Calculate the coordinates of the grid cell
            cell_low_lat = low_lat + i * step_size
            cell_low_long = low_long + j * step_size
            cell_high_lat = cell_low_lat + step_size
            cell_high_long = cell_low_long + step_size
            # Add the cell to the grid
            grid.append((cell_low_lat, cell_low_long, cell_high_lat, cell_high_long))
    return grid

In [21]:
def divide_area_in_grid(low_lat, low_long, high_lat, high_long, step_size):
    grid = []
    lat = low_lat
    while lat < high_lat:
        long = low_long
        while long < high_long:
            cell = (lat, long, min(lat + step_size, high_lat), min(long + step_size, high_long))
            grid.append(cell)
            long += step_size
        lat += step_size
    return grid

In [24]:
grid = divide_area_in_grid(low_lat, low_long, high_lat, high_long, 0.01)
print(grid)

[(48.5805798, 9.1144852, 48.5905798, 9.1244852)]


In [25]:
def divide_area_in_grid(low_lat, low_long, high_lat, high_long, step_size):
    """
    Divides a rectangular geographic area into a grid of smaller rectangles.

    Args:
        low_lat (float): Latitude of the southwest corner.
        low_long (float): Longitude of the southwest corner.
        high_lat (float): Latitude of the northeast corner.
        high_long (float): Longitude of the northeast corner.
        step_size (float): Size of each grid cell in degrees.

    Returns:
        list: A list of tuples, where each tuple represents the coordinates 
              of a grid cell as (low_lat, low_long, high_lat, high_long).
    """
    grid = []
    lat = low_lat
    while lat < high_lat:
        long = low_long
        while long < high_long:
            # Ensure cells do not exceed the high boundaries
            cell_high_lat = min(lat + step_size, high_lat)
            cell_high_long = min(long + step_size, high_long)
            
            # Add the cell to the grid
            cell = (lat, long, cell_high_lat, cell_high_long)
            grid.append(cell)
            
            # Increment longitude
            long += step_size
        
        # Increment latitude
        lat += step_size
    return grid

In [26]:
grid = divide_area_in_grid(low_lat, low_long, high_lat, high_long, 0.01)
print(grid)

[(48.5805798, 9.1144852, 48.5905798, 9.1244852)]


In [9]:
gmaps = googlemaps.Client(key=api_key) # use googlemaps library
geocode_result = gmaps.geocode('tübingen') # geocode the adress
print(geocode_result)

[{'address_components': [{'long_name': 'Tübingen', 'short_name': 'Tübingen', 'types': ['locality', 'political']}, {'long_name': 'Tübingen', 'short_name': 'Tübingen', 'types': ['administrative_area_level_3', 'political']}, {'long_name': 'Tübingen', 'short_name': 'TÜ', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Baden-Württemberg', 'short_name': 'BW', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Germany', 'short_name': 'DE', 'types': ['country', 'political']}, {'long_name': '72', 'short_name': '72', 'types': ['postal_code', 'postal_code_prefix']}], 'formatted_address': '72 Tübingen, Germany', 'geometry': {'bounds': {'northeast': {'lat': 48.5937085, 'lng': 9.131081600000002}, 'southwest': {'lat': 48.4505798, 'lng': 8.9644852}}, 'location': {'lat': 48.5216364, 'lng': 9.0576448}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 48.5937085, 'lng': 9.131081600000002}, 'southwest': {'lat': 48.4505798, 'lng': 8.9644852}}}, 'p

In [10]:
high_lat = geocode_result[0]['geometry']['bounds']['northeast']['lat']
high_long = geocode_result[0]['geometry']['bounds']['northeast']['lng']
low_lat = geocode_result[0]['geometry']['bounds']['southwest']['lat']
low_long = geocode_result[0]['geometry']['bounds']['southwest']['lng']

step_size = 0.01
grid = divide_area(low_lat, low_long, high_lat, high_long, step_size)

In [11]:
def get_dataset(low_lat, low_long, high_lat, high_low):
    """
    Calls the Google Places Text Search API to fetch restaurants in the specified area.
    
    Args:
        low_lat (float): The latitude of the southwest corner of the rectangle.
        low_long (float): The longitude of the southwest corner of the rectangle.
        high_lat (float): The latitude of the northeast corner of the rectangle.
        high_long (float): The longitude of the northeast corner of the rectangle.        

    Returns:
        list: A list of places found in the specified area.
        
    """

    # Base URL
    base_url = "https://places.googleapis.com/v1/places:searchText"
    
    # Headers
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "places.id,places.displayName,nextPageToken" 
        #"X-Goog-FieldMask": ",".join(field_textsearch) 
    }

    results = [] # List to store the results
    page_token = None  # Initialize the page token to None

    while True:
        # JSON request body
        body = {
            "includedType": "restaurant",  # Restrict to restaurants only
            "strictTypeFiltering": True,  # Only return results of the specified type
            "textQuery": "restaurant",
            "pageSize": 20,  # max results per page
            "pageToken": page_token,  # Next page token, if any
            "languageCode": "en",  # Language for results
            "locationRestriction": {
                "rectangle": {
                    "low": {"latitude": low_lat, "longitude": low_long},
                    "high": {"latitude": high_lat, "longitude": high_low}
                }}
        }

        # Send the POST request to the API
        response = requests.post(base_url, headers=headers, json=body)
        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()

            # Check if "places" key is present in the response
            if "places" in data:
                results.extend(data["places"])
            else: break # Break the loop if no results are available

            # Check if there is a next page token
            if "nextPageToken" in data:
                page_token = data["nextPageToken"]
            else:
                break
        else:
            print("Error:", response.status_code, "Message:", response.text)
            break
    
    return results

In [12]:
def fetch_restaurants_in_grid(low_lat, low_long, high_lat, high_long):
    """
    Fetch restaurants for a grid cell and whenever exactly 60 results are returned recursively subdivide cells.

    Args:
        low_lat (float): The latitude of the southwest corner of the rectangle.
        low_long (float): The longitude of the southwest corner of the rectangle.
        high_lat (float): The latitude of the northeast corner of the rectangle.
        high_long (float): The longitude of the northeast corner of the rectangle.
        step_size (float): Step size for dividing overcrowded cells.

    Returns:
        list: A list of unique restaurants found in the area.
    """
    restaurants = get_dataset(low_lat, low_long, high_lat, high_long)

    # If exactly 60 results are found, divide the cell further
    if len(restaurants) == 60:
        print(f"Overcrowded cell detected: {low_lat}, {low_long}, {high_lat}, {high_long}")
        mid_lat = (low_lat + high_lat) / 2
        mid_long = (low_long + high_long) / 2
        
        # Subdivide into four smaller cells
        sub_cells = [
            (low_lat, low_long, mid_lat, mid_long),  # Bottom-left
            (low_lat, mid_long, mid_lat, high_long),  # Bottom-right
            (mid_lat, low_long, high_lat, mid_long),  # Top-left
            (mid_lat, mid_long, high_lat, high_long)  # Top-right
        ]

        # Recursively fetch restaurants in the subdivided cells
        subdivided_restaurants = []
        for cell in sub_cells:
            subdivided_restaurants.extend(fetch_restaurants_in_grid(*cell))
        
        return subdivided_restaurants

    return restaurants

def remove_duplicates(restaurants):
    """
    Remove duplicate restaurants from the list based on unique identifiers (e.g., place_id).
    """
    unique_restaurants = {restaurant['id']: restaurant for restaurant in restaurants}
    return list(unique_restaurants.values())

In [13]:
# Loop through the grid and fetch restaurants
all_restaurants = []
for cell in grid:
    low_lat, low_long, high_lat, high_long = cell
    # Use recursive function to ensure all restaurants are fetched
    restaurants = fetch_restaurants_in_grid(low_lat, low_long, high_lat, high_long)
    all_restaurants.extend(restaurants)
    time.sleep(2)  # Sleep for 2 seconds to avoid rate limiting

# Remove duplicates from the final list
unique_restaurants = remove_duplicates(all_restaurants)
print(f"Total unique restaurants found: {len(unique_restaurants)}")

Total unique restaurants found: 213


In [ ]:
all_restaurants = remove_duplicates(all_restaurants)

SyntaxError: invalid syntax (3159705802.py, line 2)